In [16]:
import os
import re
import numpy as np 
import shutil
from sklearn.metrics import accuracy_score

import transformers
from transformers import BertTokenizer, BertModel

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
# device = 'cuda' if cuda.is_available() else 'cpu'
device = 'cpu'
import pickle
import re
# from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
# nltk.download('stopwords')

device

'cpu'

In [17]:
with open('cozmo_filtered_data.pkl', "rb") as fh:
    cozmo_data = pickle.load(fh)
with open('misty_filtered_data.pkl', "rb") as fh:
    misty_data = pickle.load(fh)

In [18]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#     text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [19]:
cozmo_data['Description'] = cozmo_data['Description'].apply(clean_text)

In [20]:
misty_data['Description'] = misty_data['Description'].apply(clean_text)

In [21]:
class MultiLabelDataset(torch.utils.data.Dataset):

    def __init__(self, text, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.targets = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [260]:
class BERTClass(torch.nn.Module):
    def __init__(self, NUM_OUT):
        super(BERTClass, self).__init__()
                   
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 256)
        self.classifier = torch.nn.Linear(256, NUM_OUT)
        self.dropout = torch.nn.Dropout(0.5)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.softmax(output)
        return output

In [331]:
def loss_fn(outputs, targets):
#     return torch.nn.CrossEntropyLoss()(outputs, targets)
    return torch.nn.BCELoss()(outputs.float(), targets.float())
#     return torch.nn.KLDivLoss()(outputs.float(), targets.float())

def train(model, training_loader, optimizer):
    model.train()
    for data in tqdm(training_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
#         _, targets = targets.max(dim=1)
#         targets = torch.autograd.Variable(targets)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()
    return loss
    
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for data in tqdm(testing_loader):
            targets = data['targets']
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            outputs = torch.sigmoid(outputs).cpu().detach()
            fin_outputs.extend(outputs)
            fin_targets.extend(targets)
    return torch.stack(fin_outputs), torch.stack(fin_targets)

In [332]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

MAX_LEN = 16
BATCH_SIZE = 32
EPOCHS = 100
NUM_OUT = 8
LEARNING_RATE = 2e-05

In [333]:
cols = ['Interst/Alarm','Confusion/Understanding','Frusteration/Relief','Sorrow/Joy','Anger/Gratitude','Fear/Hope','Boredom/Surprise','Disgust/Desire']

In [334]:
a = np.array([1,3,3,5]) < 2
b = np.array([1,3,3,5]) > 4

a | b

array([ True, False, False,  True])

In [335]:
def to_binary(data):
    x = []
    y = []
    for i in range(0,len(data)):
        a = data[cols].values[i]
        b = (a < 2)
        a = (a > 4) 
        a = a | b
        a = a * 1
        for j in range(0,len(a)):
            if a[j] == 1:
                mx = np.zeros(8)
                mx[j] = 1
                y.append(mx)
    #             print('mx', mx)
                x.append(data.Description.values[i])
#                 break
    return x, np.array(y)

# x, y =  to_binary(cozmo_data)

In [338]:
# likert_threshold = 3

# train_X = cozmo_data.Description.values
# train_y = (cozmo_data[cols].values > likert_threshold) * 1
# test_X = misty_data.Description.values
# test_y = (misty_data[cols].values > likert_threshold) * 1


train_x, train_y = to_binary(misty_data)
test_x, test_y = to_binary(cozmo_data)
# t = 642
# train_x = x[:t]
# train_y = y[:t]
# test_x = x[t:]
# test_y = y[t:]


print(len(train_x), len(train_y), len(test_x), len(test_y))
training_data = MultiLabelDataset(train_x, torch.from_numpy(train_y), tokenizer, MAX_LEN)
test_data = MultiLabelDataset(test_x, torch.from_numpy(test_y), tokenizer, MAX_LEN)

train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }    

training_loader = torch.utils.data.DataLoader(training_data, **train_params)
testing_loader = torch.utils.data.DataLoader(test_data, **test_params)

742 742 528 528


In [339]:
model = BERTClass(NUM_OUT)
model.to(device)    

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    loss = train(model, training_loader, optimizer)
    print(f'Epoch: {epoch}, Loss:  {loss.item()}') 
    guess, targs = validation(model, testing_loader)
#     guess = (guess > 0.56) * 1
    guess = guess.numpy().argmax(1)
    targs = targs.numpy().argmax(1)
    print('guess', guess)
    print('targs', targs)
    print('arracy on test set {}'.format(accuracy_score(guess, targs)))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-331-72ac498afc0f>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.t

<ipython-input-21-2f21d2e4cfd2>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'targets': torch.tensor(self.targets[index], dtype=torch.float)



Epoch: 0, Loss:  0.3639977276325226


<ipython-input-331-72ac498afc0f>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm(testing_loader):



guess [6 6 0 0 0 0 0 0 0 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0
 0 6 0 0 0 0 0 0 0 0 6 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 6 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 6 0 0 6 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 6 0 0 0 6 0 0 0
 0 0 0 0 0 0 0 6 0 0 6 0 0 0 0 6 0 6 0 0 6 0 0 0 0 6 0 0 0 0 0 0 0 6 6 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 6 0 0 0 0 0 6 0 0 0 0 1 0 0 0 0 6
 0 0 1 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 6 6 0 0 6 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 6 6 0 0 0 0 0 0 0 0 0 6 0 0 0 6 0
 6 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0
 0 0 0 0 0 0 6 0 0 0 0 0 0 0 6 0 0 0 0 6 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0

...


guess [1 0 0 1 0 0 7 1 0 6 4 0 1 7 0 1 0 1 3 6 0 1 0 0 7 0 0 5 1 0 3 7 1 0 0 0 1
 0 0 1 0 1 4 1 6 6 3 7 7 6 0 1 0 1 0 7 3 1 0 0 6 6 0 0 0 1 1 0 0 1 0 1 1 0
 0 1 0 1 0 7 0 0 3 0 3 1 6 0 6 1 1 3 7 3 0 6 7 0 6 1 1 1 6 4 7 0 1 7 6 6 0
 0 6 0 0 0 0 1 1 0 4 1 0 0 1 7 1 0 0 0 4 4 0 0 0 7 1 6 2 1 0 7 0 0 1 0 6 0
 6 0 7 5 1 0 1 1 0 6 0 4 0 1 0 0 0 0 1 1 1 0 3 7 0 1 0 4 3 6 0 7 7 3 0 0 3
 3 1 6 0 0 7 0 1 0 0 1 0 0 0 1 1 4 7 1 4 0 0 1 0 1 0 4 0 1 6 1 1 7 0 1 6 0
 1 0 0 1 6 1 1 0 3 0 0 6 0 7 1 0 1 0 4 1 0 0 0 3 0 3 0 1 0 0 1 1 1 3 0 6 1
 0 0 0 0 3 6 0 7 0 6 0 0 3 0 3 1 0 0 4 0 3 0 1 7 3 7 0 7 7 1 0 6 1 4 1 7 1
 6 0 7 0 7 1 1 0 3 0 0 6 0 1 7 6 0 0 0 0 1 1 6 0 0 0 0 3 0 0 1 6 0 0 0 0 6
 1 6 1 0 1 1 7 0 0 6 0 4 1 1 7 0 3 0 1 6 7 0 0 1 0 0 7 0 0 3 1 1 1 6 1 3 6
 0 0 0 0 0 1 7 0 1 6 6 0 0 1 0 3 0 1 0 0 7 1 5 0 0 6 1 3 1 0 0 0 0 0 0 0 0
 3 1 0 0 3 0 3 7 0 0 3 0 0 3 1 0 0 0 0 0 1 0 1 0 0 6 0 1 3 0 3 0 6 1 0 4 7
 0 7 1 7 0 0 3 1 0 0 0 0 7 0 1 0 0 0 1 6 7 0 6 0 0 1 0 6 0 7 0 0 0 1 0 3 1
 0 6 0 7 7 1 1 1 6


Epoch: 98, Loss:  0.24917198717594147



guess [0 6 1 0 6 7 1 1 0 1 1 0 0 3 1 6 0 1 1 0 0 0 4 0 6 6 0 1 0 6 1 0 3 0 4 0 4
 0 7 0 0 7 0 6 0 0 1 0 0 6 6 0 0 0 0 0 1 0 6 6 0 7 4 0 1 0 3 0 0 0 0 0 3 0
 3 6 6 1 0 1 6 0 0 0 6 1 4 0 1 4 7 6 0 7 3 3 0 6 0 3 0 6 7 2 3 0 1 4 1 6 0
 6 1 0 4 6 0 0 0 0 0 0 1 3 0 0 1 0 0 0 6 0 0 0 6 6 6 0 0 1 6 0 6 0 0 0 1 6
 2 0 0 0 0 0 1 0 0 0 4 1 0 4 6 0 0 0 0 0 0 0 0 0 1 0 1 0 0 6 0 6 0 0 2 1 4
 0 0 0 0 6 7 0 0 6 6 6 3 6 6 0 0 0 0 0 6 0 3 0 0 1 6 0 3 0 0 0 3 0 0 1 0 3
 0 1 1 6 0 0 0 1 0 1 0 3 3 6 0 0 0 0 0 0 1 0 0 0 0 6 1 0 0 0 3 1 6 0 0 3 0
 0 1 7 0 0 0 0 0 0 6 6 3 7 0 6 3 3 7 1 0 0 6 3 0 6 0 3 0 0 3 0 0 0 3 0 1 1
 0 0 0 1 1 0 0 1 1 0 0 6 0 0 0 1 6 1 0 0 0 0 1 0 0 0 0 1 1 6 0 0 0 6 6 3 1
 4 6 1 1 3 3 0 1 1 0 1 0 0 0 0 0 7 0 6 3 0 0 0 1 0 0 1 1 3 0 0 0 0 6 7 1 0
 3 0 6 0 6 6 0 0 0 3 0 0 4 0 0 3 0 6 6 0 0 0 6 0 4 0 0 1 0 3 3 1 4 1 0 0 0
 1 1 0 7 1 0 6 0 0 0 0 6 0 3 0 1 0 0 3 0 0 0 1 1 3 0 7 6 3 3 0 1 1 3 0 0 3
 1 0 1 3 6 0 1 1 1 0 1 0 3 1 3 0 3 0 0 0 0 0 0 0 0 1 6 3 0 0 0 6 1 6 6 0 0
 0 4 0 3 3 0 0 0 0


Epoch: 99, Loss:  0.2897215187549591



guess [0 0 3 7 3 6 0 0 0 1 0 0 1 0 0 6 1 0 6 6 1 0 1 0 3 1 0 0 0 1 3 7 0 0 4 7 0
 7 0 1 6 0 1 0 7 1 0 3 0 1 0 3 0 1 6 6 6 0 1 7 0 3 0 1 1 0 0 0 0 0 0 6 0 0
 3 0 6 3 0 0 1 0 4 0 0 0 0 0 0 6 0 3 0 0 7 3 0 6 0 7 1 4 1 6 6 6 6 3 1 0 3
 7 0 0 3 3 0 0 0 0 7 1 1 0 6 0 1 0 3 0 3 6 3 3 0 0 0 0 0 0 3 4 3 3 6 0 1 3
 0 0 3 0 1 6 1 0 6 0 1 6 0 1 1 3 0 0 0 1 0 1 3 0 3 1 3 0 0 0 0 4 0 0 6 1 1
 0 6 3 1 1 3 1 1 2 0 0 0 3 1 0 0 1 2 0 0 3 0 7 0 3 1 0 3 0 1 0 0 0 3 0 3 1
 0 6 0 1 1 0 0 1 0 3 1 3 3 0 1 0 0 0 0 0 1 3 0 0 1 0 0 0 6 0 0 0 0 6 0 6 6
 1 0 1 0 0 7 4 0 4 0 1 1 3 0 6 1 0 0 1 6 2 1 3 3 6 1 0 1 1 3 0 3 1 0 1 7 6
 0 3 0 3 6 3 0 1 0 3 0 0 1 0 0 0 0 1 7 6 6 7 0 0 0 1 1 0 3 1 3 0 1 0 1 3 0
 0 0 0 0 0 4 0 0 0 0 7 0 3 4 1 1 0 6 0 0 1 0 3 0 0 1 1 3 0 3 1 6 4 0 0 0 1
 1 0 0 1 0 3 0 3 1 0 3 3 0 0 0 0 0 0 0 0 0 0 1 4 0 6 0 6 1 3 3 0 0 0 0 0 0
 0 0 0 6 0 6 0 0 0 0 1 0 4 0 0 0 0 1 0 1 0 0 4 0 3 7 0 0 0 6 6 0 0 0 0 0 3
 7 1 1 1 0 1 7 6 0 0 0 0 4 0 0 7 1 3 0 0 0 0 0 6 0 0 0 3 1 0 0 3 4 7 1 0 0
 0 6 6 0 0 0 3 1 0